In [133]:
%pip install openai python-dotenv
%pip install pandas



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [134]:
import os
import getpass
import json
import openai
from collections import defaultdict
import numpy as np 
from rich.console import Console 
from rich.table import Table

In [135]:
import pandas as pd
import openai
import json
import ast
from collections import OrderedDict
import re



# Set API key
openai.api_key = "sk-V6AYRLCSNq5CM8uc1wdYjFEc5cedfEfwusaw3bntbFT3BlbkFJ_uLzkauLWwo2p-9RA1xQqiaDI-_DS72gElgel1PJkA"


# Function to get a completion
def get_completion(messages, model="gpt-4"):
   response = openai.ChatCompletion.create(
       model=model,
       messages=messages,
       temperature=0,
   )
   return response['choices'][0]['message']['content']


# Function to read messages from a CSV file into a DataFrame
def read_messages_from_csv(file_path):
   return pd.read_csv(file_path)


# File path to CSV file
file_path = '/Users/katherineyang/Downloads/unlabeled.csv'
labeled_file_path = '/Users/katherineyang/Downloads/labeled.csv'


# Read messages into a DataFrame
df = read_messages_from_csv(file_path)
labeled_df = read_messages_from_csv(labeled_file_path)

# Initialize the system message
system_message = {
   "role": "system",
   "content": """You are a meticulous conversation labeller. 
          Your job is to accurately label who is the intended recipient of each message in a transcript, based on preceding context.

   - If the message is directed towards a single participant, label the recipient with their ID. 
   - If the message is directed towards the group, then label the recipient as "all."
   - If the message is directed towards a sequence of participants, label the recipient with the list of IDs: e.g. 2,3. or All, 3, 4.

   
  Return a list of dictionaries in valid JSON:
    ```
    [
      {
        "sender": [Sender ID]
        "message": [First few words of the message]
        "recipient": [Labeled recipient(s)]
      },
      {
        "sender": [Sender ID]
        "message": [First few words of the message]
        "recipient": [Labeled recipient(s)] 
      }
    ]
    ```
      

   EXAMPLES
  1a. Single Participant
    - Vero (ID 0) asks Nathan (ID 281):
      ```
      {
        "sender": "Vero"
        "message": "Nathan, could you clarify the item we discussed?"
        "recipient": 281
      }
      ```
    - Nathan responds to Vero:
      ```
      {
        "sender": "Participant 281"
        "message": "Sure, we agreed on the pistol and jack knife."
        "recipient": 0
      }
      ```
    1b. Single Participant
    - A participant expresses agreement with another person's idea, usually by saying "Yeah..." 
      ```
      {
        "sender": Participant 226
        "message": "I think the parachute would be a good item to have."
        "recipient": all
      }
      ```
    - Participant 840 responds to Participant 226
      ```
      {
        "sender": Participant 840
        "message": "Yeah, it's got bright colors and there's a lot we can do with it."
        "recipient": 226
      }
      ```
    
      1c. Single Participant
    - A participant expresses agreement with another person's idea, usually by saying "Yeah, I also had..." 
      ```
      {
        "sender": Participant 226
        "message": "Maybe the pistol would be more helpful."
        "recipient": all
      }
      ```
    - Participant 840 responds to Participant 226
      ```
      {
        "sender": Participant 840
        "message": "Yeah, I also had the pistol in my top five."
        "recipient": 226
      }
      ```

    1d. Single Participant
    - A participant expresses agreement with another person's idea by saying "Let's do that".
      ```
      {
        "sender": Participant 226
        "message": "I think the pistol should be in the top five."
        "recipient": all
      }
      ```
    - Participant 840 responds to Participant 226
      ```
      {
        "sender": Participant 840
        "message": "Let's do that."
        "recipient": 226
      }
      ```

    1e. Single Participant
    - A participant expresses agreement with another person's idea by saying "Sounds good".
      ```
      {
        "sender": Participant 226
        "message": "So maybe we can put pistol in the top five?"
        "recipient": all
      }
      ```
    - Participant 840 responds to Participant 226
      ```
      {
        "sender": Participant 840
        "message": "Sounds good."
        "recipient": 226
      }
      ```

      1f. Single Participant
    - A participant adds on to another participant's idea, usually by saying "And..."
      ```
      {
        "sender": Participant 251
        "message": "I think the coat is important because it can insulate us if it gets cold."
        "recipient": all
      }
      ```
    - Participant 165 responds to Participant 251
      ```
      {
        "sender": Participant 165
        "message": "And it can block the sun during the day."
        "recipient": 251
      }
      ```


  2. All Participants
    - A participant addresses everyone:
      ```
      {
        "sender": Participant 102
        "message": "I think we should all meet tomorrow."
        "recipient": all
      }
      ```


  3. **Multiple Participants in Sequence**:
    - A participant addresses multiple participants:
      ```
      {
        "sender": Participant 103
        "message": "Sarah, can you handle the report? And John, please check the budget."
        "recipient": 104,105
      }
      ```


  4. **Mixed Addressing**:
  1a.
    - A participant starts with a general statement and then addresses a specific participant:
      ```
      {
        "sender": Participant 104
        "message": "Everyone, great job on the project. John, can you send the final report?"
        "recipient": all,105
      }
      ```
  1b.
    - A participant adds on to another participant's idea, and then addresses the whole group:
      ```
      {
        "sender": Participant 104
        "message": "I think the coat is important because it can insulate us if it gets cold."
        "recipient": all
      }
      ```
    - Participant 105 responds to Participant 104
      ```
      {
        "sender": Participant 105
        "message": "And it can help us carry things. So let's put it in the top five?"
        "recipient": 104, all
      }
      ```


**Additional Clarification**:
- Do not use follow-up messages to determine the receiver of the current message. In a group meeting, one participant may ask a question to the entire group, but only one may respond. Base the receiver labeling on the current and preceding messages only. For example this is an example of a message sent by Participant 102 was directed to all participants even though Participant 105 responded directly:
 ```
- Message directed to all participants
 Sender: Participant 102
 Message: "Are we done with this task?"
 Receiver: all


- Participant 105 responds to the message
 Sender: Participant 105 
 Message: "Not yet."
 Receiver: Participant 102
 ```
** Note: it would be incorrect to label the intended recipient of the first message above as Participant 105, even though they responded to it, as it is biased to use future messages to label previous ones and the message was directed to all participants, rather than the initial responder.


**Identifying Potential Data Inaccuracies**:
- If the sender of a message appears to be highly likely incorrect due to data inaccuracy, include a standardized comment in parentheses indicating this. For example:
 ```
 Sender: Vero (possible data error)
 Message: "Vero do you know?"
 Receiver: 0
 ```

**When Vero provides a formal definition or detailed descriptions of an item, we almost always consider that as addressing “all” for simplicity.**:
- Participant 281 asks what an item is.
  ```
  {
    "sender": Participant 281
    "message": "What is a blow dart?"
    "recipient": 0
  }
  ```
- Vero responds to Participant 281
  ```
  {
    "sender": Vero
    "message": "A sharp projectile that can be shot out of a blowpipe. A blowpipe is a long tube in which an individual blows air through one end and a blow dart shoots out the other end."
    "recipient": all
  }
  ```
      
"""
}

# Initialize variables
batch_size = 15
lookback_size = 5
num_batches = len(df) // batch_size + (1 if len(df) % batch_size != 0 else 0)

previous_batch_messages = []
messages_seen = set()
result_list = []

correct_messages = 0
total_messages = 0

for batch_num in range(num_batches):
    start_idx = batch_num * batch_size
    end_idx = min(start_idx + batch_size, len(df))
    
    # Prepare messages for the current batch
    batch_messages = df.iloc[start_idx:end_idx]
    user_messages = [
        {"role": "user", "content": f"Message {start_idx + i + 1}: User: {row['user']}, Text: '{row['text']}'"}
        for i, row in batch_messages.iterrows()
    ]
    
    # Combine previous context and current batch messages
    context = previous_batch_messages + [system_message]
    current_conversation = context + user_messages
    
    response = get_completion(current_conversation)
    
    if response:
        # Extract the JSON-like responses from the output
        labels = response.replace("\n", " ")
        labels = ast.literal_eval(labels)

        # Convert each dictionary in labels to tuples and add to a set
        labels_tuple = [tuple(item.items()) for item in labels]
    
        # Use OrderedDict to get tuples in order and remove duplicates
        unique_labels = list(OrderedDict.fromkeys(labels_tuple))

        index = 0
        output_labels = []
        # Process each unique label
        for i,label in enumerate(unique_labels):
            if label[1][1] not in messages_seen:
                messages_seen.add(label[1][1])
                result_list.append(label) # for the csv file
                total_messages += 1
                
                # Measuring accuracy:
                # Extract the exact label
                output_label = label[2][1]
                if output_label == "Vero":
                   output_label == "0"
                str_output_label = str(output_label).strip()
                if re.search(r'Participant', str_output_label, re.IGNORECASE):
                # split string 
                  split_string = str_output_label.split()
                  second_element = split_string[1]
                  print("LABEL: ", second_element)
                  if second_element == "Vero":
                     second_element == "0"
                  # append to output labels
                  output_labels.append(second_element)
                else:
                  output_labels.append(output_label)

                # Get the correct label from the CSV
                correct_label = labeled_df['to'].iloc[start_idx + i]
                str_correct_label = str(correct_label).strip()

                print(str_correct_label)
                print(output_labels[-1])

                if str_correct_label == str(output_labels[-1]):
                    correct_messages += 1
                    print(f"Correct match for message {total_messages}: {output_label}")
                else:
                    print(f"Mismatch for message {total_messages}: Expected {correct_label}, Got {output_label}")
            
    # Update previous_batch_messages with the last `lookback_size` messages from the current batch
    previous_batch_messages = user_messages[-lookback_size:] if batch_num > 0 else []

# Calculate accuracy
accuracy = correct_messages / total_messages 
print(f"Accuracy: {accuracy:.2%}")

# convert result_list into a csv file
formatted_list = []
for idx, item in enumerate(result_list):
    sender = item[0][1]
    message = item[1][1]
    recipient = item[2][1]
    formatted_message = {
        "role": "user",
        "content": f"Message {idx + 1}: User: {sender}, Text: '{message}', Recipient: {recipient}"
    }
    formatted_list.append(formatted_message)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(formatted_list)

# Save the DataFrame to a CSV file
output_file = "formatted_messages.csv"
df.to_csv(output_file, index=False)

print(f"CSV file '{output_file}' has been created successfully.")



0
0
Correct match for message 1: 0
all
all
Correct match for message 2: all
0
0
Correct match for message 3: 0
all
all
Correct match for message 4: all
226
226
Correct match for message 5: 226
all
all
Correct match for message 6: all
226
226
Correct match for message 7: 226
all
all
Correct match for message 8: all
840
840
Correct match for message 9: 840
685
685
Correct match for message 10: 685
all
all
Correct match for message 11: all
226
226
Correct match for message 12: 226
all
all
Correct match for message 13: all
226
226
Correct match for message 14: 226
685
685
Correct match for message 15: 685
all
0
Mismatch for message 16: Expected all, Got 0
0
0
Correct match for message 17: 0
all
all
Correct match for message 18: all
0
0
Correct match for message 19: 0
all
all
Correct match for message 20: all
226
226
Correct match for message 21: 226
685
685
Correct match for message 22: 685
all
all
Correct match for message 23: all
all
all
Correct match for message 24: all
226
226
Correct 

In [136]:
# ChatCompletion(id='chatcmpl-9thkIwKsbiJi24QSXwxU8lWKcbm8i', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sender: Vero                                         Message: 'Nathan, could you clarify the item we discussed?'                                         Receiver: Nathan", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723063226, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=23, prompt_tokens=207, total_tokens=230))